# Notebook that reads in GPM rainfall data and extracts data for cells in the Philippines

In [ ]:
%load_ext jupyter_black
# libraries
import pandas as pd
import numpy as np
import os
import geopandas as gpd
import rasterio
from rasterstats import zonal_stats
from pathlib import Path

In [ ]:
# Setting directory
input_dir = (
    Path(os.getenv("STORM_DATA_DIR"))
    / "analysis/02_new_model_input/03_rainfall/input"
)

# grid
grid_name = "analysis/02_new_model_input/02_housing_damage/output/phl_0.1_degree_grid_land_overlap.gpkg"
grid_file = Path(os.getenv("STORM_DATA_DIR")) / grid_name
grid = gpd.read_file(grid_file)

# gpm data
gpm_file_name = "gpm_data/rainfall_data/output_hhr/"
gpm_folder_path = Path(input_dir, gpm_file_name)
typhoon_list = os.listdir(gpm_folder_path)

# outputs
processed_output_dir = Path(
    input_dir / "gpm_data/rainfall_data/output_hhr_processed/"
)
output_dir = (
    Path(os.getenv("STORM_DATA_DIR"))
    / "analysis/02_new_model_input/03_rainfall/output"
)

Testing raster method

In [ ]:
# Testing the raster stats method
day_list = os.listdir(gpm_folder_path / typhoon_list[1] / "GPM")
file_list = os.listdir(gpm_folder_path / typhoon_list[1] / "GPM" / day_list[0])
file = Path(
    gpm_folder_path / typhoon_list[1] / "GPM" / day_list[0] / file_list[0]
)
input_raster = rasterio.open(file)
array = input_raster.read(1)
# checking if crs are the same
input_raster.crs == grid.crs

In [ ]:
# computing stats for 4 adjacent cells
summary_stats = zonal_stats(
    grid,
    array,
    stats=["min", "max", "mean", "std"],
    nodata=29999,
    all_touched=True,
    affine=input_raster.transform,
)
grid_stats = pd.DataFrame(summary_stats)

In [ ]:
## Using OCHA AnticiPy for zonal stats
## Install via pip install ocha-anticipy
# More information here: https://github.com/OCHA-DAP/ocha-anticipy
# https://ocha-anticipy.readthedocs.io/en/latest/introduction.html
# Takes longer than zonal stats

import ochanticipy
import rioxarray as rio

# opening raster and clipping it to use only grid extents and extracting stats
input_raster = rio.open_rasterio(file)
rast_clip = input_raster.rio.clip_box(*grid.total_bounds)
grid_df = rast_clip.oap.compute_raster_stats(
    gdf=grid, feature_col="Centroid", all_touched=True
)

In [ ]:
# Writing loop using rio.clip
# Takes the longest time out of the 3 methods
grid_vals = pd.DataFrame(
    {"id": grid["id"], "Centroid": grid["Centroid"], "mean": None, "max": None}
)
for grd in grid.Centroid:
    grd_sel = grid[grid.Centroid == grd]
    grid_mean = rast_clip.rio.clip(grd_sel["geometry"], all_touched=True)
    grid_vals.loc[grd_sel.index.values, ["mean"]] = grid_mean.mean().values
    grid_vals.loc[grd_sel.index.values, ["max"]] = grid_mean.max().values

# Will proceed with raster stats as it seems to be the fastest

Looping over typhoons

This section takes a very long time to process.

In [ ]:
# setting up loop for running through all typhoons
# extracting the max and mean of the 4 adjacent cells due to shifting to grids
stats_list = ["mean", "max"]
for typ in typhoon_list:
    day_list = os.listdir(gpm_folder_path / typ / "GPM")
    day_df = pd.DataFrame()
    for day in day_list:
        file_list = os.listdir(gpm_folder_path / typ / "GPM" / day)
        file_df = pd.DataFrame()
        for file in file_list:
            if file.startswith("3B-HHR"):
                file_path = Path(gpm_folder_path / typ / "GPM" / day / file)
                input_raster = rasterio.open(file_path)
                array = input_raster.read(1)
                summary_stats = zonal_stats(
                    grid,
                    array,
                    stats=stats_list,
                    nodata=29999,
                    all_touched=True,
                    affine=input_raster.transform,
                )
                grid_stats = pd.DataFrame(summary_stats)
                # change values by dividing by 10 to mm/hr
                grid_stats[stats_list] /= 10
                grid_merged = pd.merge(
                    grid.drop(["geometry", "Longitude", "Latitude"], axis=1),
                    grid_stats,
                    left_index=True,
                    right_index=True,
                )
                grid_merged["start"] = "%s%s:%s%s:%s%s" % (
                    *file.split("-S")[1][0:6],
                )
                grid_merged["end"] = "%s%s:%s%s:%s%s" % (
                    *file.split("-E")[1][0:6],
                )

                file_df = pd.concat([file_df, grid_merged], axis=0)
        file_df["date"] = str(day)
        day_df = pd.concat([day_df, file_df], axis=0)
    day_df["time"] = day_df["date"].astype(str) + "_" + day_df["start"]
    for stats in stats_list:
        day_wide = pd.pivot(
            day_df,
            index=["id", "Centroid"],
            columns=["time"],
            values=[stats],
        )
        day_wide.columns = day_wide.columns.droplevel(0)
        day_wide.reset_index(inplace=True)
        day_wide.to_csv(
            processed_output_dir / str(typ + "_gridstats_" + stats + ".csv"),
            index=False,
        )